# Data management <a name="chap:R_data"></a>

<!--NAVIGATION-->

<[Back to Workshop Contents](Index.ipynb)>

This chapter aims at introducing you to key principles and methods for data processing and storage in R.

In [1]:
library(repr) 
options(repr.plot.width=6, repr.plot.height=4) # Change plot sizes (in cm)

## Data wrangling

You are likely to spend far more time than you think dredging through  your data manually, checking it, editing it, and reformatting it to  make it useful for the actual data exploration and statistical  analysis. For example, you may need to:  
* Identify the variables vs observations within the data --- somebody else might have recorded the data, or you might have  collected the data some time back!
* Fill in zeros (true measured/observed absences) 
* Identify and add a value (e.g., `-999999`) to denote missing observations	
* Derive or calculate new variables from the raw observations (e.g., convert measurements to SI units; kilograms, meters, seconds, etc.)
* Reshape/reformat your data into a layout that works best for analysis (e.g., for R itself) --- e.g., from wide to long data  format for repeated (across sites, plots, plates, chambers, etc) measures data
* Merge multiple datasets together into a single data sheet

And this is far from an exhaustive list. Doing so many different things  to your raw data is both time-consuming and risky. Why risky? Because  to err is very human, and every new, tired mouse-click and/or keyboard-stab has a high probability of inducing an erroneous data point!

*An illustration of a (metaphorical) datum being wrangled into submission:*
![An illustration of a (metaphorical) datum being wrangled into submission.](./graphics/Wrangling2.jpg)

### Some data wrangling principles

So you would like to a record of the data wrangling process (so that it  is repeatable and even reversible), and automate it to the extent  possible. To this end, here are some guidelines:

* Store data in universally-readable, nonproprietary software formats (e.g., `.csv`)
* Use plain ASCII text for your file names, variable/field/column names, and data values --- make sure the "text encoding" is correct and standard  (e.g., `UTF-8`)
* Keep a metadata file for each unique dataset (agian, in  non-proprietary format)
* Don't (over-)modify your raw data by hand --- use scripts  instead --- keep a copy of the data as they were recorded.
* Use meaningful names for your data and files and field (column) names
* When you add data, try not to add columns (widening the format); rather, design your  tables/datasheets so that you add only rows (lengthening the format) --- and convert ``wide format data'' to ``long format data'' using  scripts, not by hand!
* All cells within a data column should contain only one type of information (i.e., either text (character), numeric, etc.). 
* Ultimately, consider creating a relational database for your  data (see [below](#Databases-and-R)).

This is not an exhaustive list either --- read the [Borer et al (2009) paper](#Readings).

We will use the Pound Hill dataset collected by students in a past Silwood Field Course week for understanding/illustrating some of these  principles. 

To start with, we need to import the {\tt raw} data file, for which, follow these steps:

* Go to the repository and navigate to the `Data` directory. 
* Copy the file `PoundHillData.csv` and `PoundHillMetaData.csv` files into your own R week's `Data` directory. 
* Then load the data in R: 

In [2]:
MyData <- as.matrix(read.csv("../data/PoundHillData.csv",header = F,  stringsAsFactors = F))
MyMetaData <- read.csv("../data/PoundHillMetaData.csv",header = T,  sep=";", stringsAsFactors = F)

Note that:

Loading the data `as.matrix()`, and setting the `header` and `stringsAsFactors` guarantees that the data are imported “as is” so that you can wrangle them. Otherwise `read.csv` will convert the first row to column headers,
convert everything to factors, etc. Note that all the data will be converted to the character class in the resulting matrix called `MyData` because at least one of the entries is already character class.

For the metadata loading, the `header` is set to true because we do have metadata headers (`FieldName` and
`Description`), and I have used semicolon (`;`) as delimiter because there are commas in one of the field descriptions.

I have avoided spaces in the columns headers (so “FieldName” instead of “Field Name”) — please avoid spaces in field or column names as much a possible as R will replace each space in a column header with a dot, which may be confusing.

>*Tip:* In `R`, you can use `F` and `T` for boolean `FALSE` and `TRUE` respectively. To see this, type 
```rscript 
a <- T``` 
in the R commandline, and then see what R returns when you type `a`

We won’t do anything with the metadata file in this session except inspect the information it contains. So let's look at some of the key principles listed above.

#### Keep a metadata file for each unique dataset

Data wrangling really begins immediately after data collection. You may collect data of different kinds (e.g., diversity, biomass, tree girth), etc. Keep the original spreadsheet well documented using a “metadata” file that describes the data (you would hopefully have written the first version of this even before you started collecting the data!). The minimum information needed to make a metadata file useful is a description of each of the *fields* — the column or row headers under which the information is stored in your data/spreadsheet.

Have a look at the metadata file for the Pound Hill dataset:

In [3]:
MyMetaData

FieldName,Description
Cultivation,"Cultivation treatments applied in three months: october, may, march"
Block,Treatment blocks ids: a-d
Plot,Plot ids under each treatment : 1-12
Quadrat,Sampling quadrats (25x50 cm each) per plot: Q1--Q6
SpeciesCount,Number of individuals of species (count) per quadrat


Ideally, you would also like to add more information about the data, such as the measurement units of each type of observation. Here, there is just one type of observation: Number of individuals of species per sample (plot), which is a count (integer, or `int` class).

#### Don’t (over-)modify your raw data by hand

When the dataset is large (e.g., 1000’s of rows), cleaning and exploring it can get tricky, and you are very likely to make many mistakes. You should record all the steps you used to process it with an R script rather than risking a manual and basically irreproducible processing. Most importantly *avoid or minimize editing your raw data
file*. Let’s see how we can modify the data using `R`. In fact, we should now start to keep a record of what we are doing to the data. This is illustrated in a code data file available on the bitbucket repository.

>**Tip:** Sometimes you may run into (unexpected) bugs when importing and running scripts in `R` because your file has a no-standard text encoding. You may need to specify the encoding in that case, using the `encoding` argument of `read.csv()` and ` read.table()`. You can check the encoding of a file by using `find` in Linux/Mac. Try in your UNIX terminal: 
```bash
file -i ../data/PoundHillData.csv
``` 
or, check encoding of all files in the `Data` directory: 
```bash
file -i ../data/*.csv
```  
Use `file -I` instead of `file -i` in a Mac terminal

Now, go to the master repository and navigate to the `Code`directory, and copy the script `DataWrang.R` into your own R training directory's `Code` directory and open it.

Go through the script carefully line by line, and make sure you understand what’s going on. Read the comments — add to them if you want. One of the examples of data modification that you must avoid doing by hand, is illustrated in the script: filling in zeros.

#### Convert wide format data to long format using scripts

You typically record data in the field or experiments using a “wide” format, where a subject’s (e.g., habitat, plot, treatment, species etc) repeated responses or observations (e.g., species count, biomass, etc) will be in a single row, and each response in a separate column. The raw Pound Hill data were recorded in precisely this way. However, the wide format is not ideal for data analysis — instead you need the data in a “long” format, where each row is one observation point per subject. So each subject will have data in multiple rows. Any measures/variables/observations that don’t change across the subjects will have the same value in all the rows.

For humans, the wide format is generally more intuitive for viewing and recording (e.g., in field data sheets) since one can actually view more of the data visually. However, the long format is more machine-readable and is closer to the formatting of databases.

You can switch between wide and formats using `melt()` and `dcast()` from the `reshape2` package, as illustrated in `DataWrang.R`.

### And then came dplyr and tidyr

So if you think this is the end of the options you have for data wrangling in R, think again. There are new kids on the block: `dplyr` — the next iteration of `plyr` that addresses the speed issues in the latter, and `tidyr`,
essentially a nicer wrapper to the ` reshape2` package with additional functions.

You will have to install these packages using `sudo apt get install` in Linux or `install.packages()` across all
platforms (see the Intro to R Chapter in [these notes](https://github.com/mhasoba/TheMulQuaBio/blob/master/silbiocomp/SilBioComp.pdf)).

Together, these two packages have many many useful functions. Let’s have a quick look at `dplyr`:

In [4]:
require(dplyr)
attach(iris)
dplyr::tbl_df(iris) #like head(), but nicer!
dplyr::glimpse(iris) #like str(), but nicer!
utils::View(iris) #same as fix()!
dplyr::filter(iris, Sepal.Length > 7) #like subset(), but nicer!
dplyr::slice(iris, 10:15) # something new!

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa
4.9,3.1,1.5,0.1,setosa


Observations: 150
Variables: 5
$ Sepal.Length <dbl> 5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4,...
$ Sepal.Width  <dbl> 3.5, 3.0, 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1, 3.7,...
$ Petal.Length <dbl> 1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.4, 1.5, 1.4, 1.5, 1.5,...
$ Petal.Width  <dbl> 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2, 0.2, 0.1, 0.2,...
$ Species      <fct> setosa, setosa, setosa, setosa, setosa, setosa, setosa...


Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
7.1,3.0,5.9,2.1,virginica
7.6,3.0,6.6,2.1,virginica
7.3,2.9,6.3,1.8,virginica
7.2,3.6,6.1,2.5,virginica
7.7,3.8,6.7,2.2,virginica
7.7,2.6,6.9,2.3,virginica
7.7,2.8,6.7,2.0,virginica
7.2,3.2,6.0,1.8,virginica
7.2,3.0,5.8,1.6,virginica
7.4,2.8,6.1,1.9,virginica


Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
4.9,3.1,1.5,0.1,setosa
5.4,3.7,1.5,0.2,setosa
4.8,3.4,1.6,0.2,setosa
4.8,3.0,1.4,0.1,setosa
4.3,3.0,1.1,0.1,setosa
5.8,4.0,1.2,0.2,setosa


Note that the double colon (`::`) notation of `dplyr` and ` tidyr` is like the dot notation in `python` — it allows you to access a particular function from these packages. So, for instance, if you want to use `tbl_df()` from `dplyr`, the command syntax would be `dplyr::tbl_df(MyData)`. This new syntax is basically to avoid conflict in names of functions in by these new packages with the function names that already exist in the base R packages. For example,the `dplyr` function `filter` already exists in the base R package `stats`. Thus, when you first load
`dplyr`:

In [5]:
library(dplyr)

you get:

```rscript 
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union
```

Learning to use `ddply` and `tidyr` involves learning some new syntax and a lot of new commands, but if you plan to
do a lot of data wrangling, you may want to get to know them well. Have a look at [this website](https://blog.rstudio.org/2014/01/17/introducing-dplyr) and this [cheatsheet](https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf),
also available in the [Readings directory]() under `DataDataData!` in the course bitbucket repository.

### On to data exploration

Once you have wrangled the Pound Hill data to its long format, you are ready to go! You may want to start by examining and visualizing the basic properties of the data, such as the number of tree species (41) in the dataset, number of quadrats (replicates) per plot and cultivation treatment, etc.

The first visulatization you can try is a histogram of abundances of species, grouped by different factors. For example, you can look at distributions of species’ abundances grouped by `Cultivation`).

### Exercises

(a) We used `reshape2` in `DataWrang.R` for wrangling that dataset. Write a new script called `DataWrangTidy.R` that uses `dplyr` and `tidyr` for the same data wrangling steps. The best way to do this is to copy `DataWrang.R` and rename it `DataWrangTidy.R`. Then systematically redo the script from start to end, looking for a function in `dplyr` and ` tidyr` that does the same thing in each wrangling step.

For example, to convert from wide to long format, instead of using ` melt()` (or `dcast()`) from the `reshape2` package, you can use `gather()` from `tidyr`.

Don’t forget the [cheatsheet](
https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf) 
(also available in the Readings directory under `DataDataData!` in the course bitbucket repository).

## Handling Big Data in R

The buzzword ‘Big Data’ basically refers to datasets that have the following properties:

1.  A dataset that does not fit into available RAM on one system (say, >2 gigabytes).

2.  A dataset that has so many rows (when in it’s long format — see above sections) that it significantly slows down your analysis or simulation without vectorization (that is, when looping).

Both these criteria are programming language- and computer hardware-dependent, of course. For example, a 32-bit OS can only handle ~2 GB of RAM, so your computer screams “Big Data!” (slows down/hangs) every time you try to handle a dataset in that range.

R reads data into RAM all at once when you using the `read.table` (or its wrapper, `read.csv()` — maybe
you have realized by now that `read.csv()` is basically calling `read.table()` with a particular set of options. That is, objects in R live in memory entirely, and big-ish data in RAM will cause R to choke. Python has similar problems, but you can circumventthese to an extent by using `numpy` arrays (see the basic Python Chapter in [these notes](https://github.com/mhasoba/TheMulQuaBio/blob/master/silbiocomp/SilBioComp.pdf)).

There are a few options (which you can combine, of course) if you are actually using datasets that are so large:

* Import large files smartly; e.g., using `scan()` in R, and then create subsets of the data that you need. Also, use the `reshape` or `tidyr` package to covert your data in the most “square” (so neither too long or too wide) format as possible. Of course, you will need subsets of data in long format for analysis (see sections above).

* Use the `bigmemory` package to load data in the gb range (e.g., use `read.big.matrix()` instead of `read.table()`. This package also has other useful functions, such as `foreach()` instead of `for()` for better memory management.

* Use a 64 bit version of R with enough memory and preferably on Linux!

* Vectorize your analyses/simulations to the extent possible (See Python and R chapters in [these notes](https://github.com/mhasoba/TheMulQuaBio/blob/master/silbiocomp/SilBioComp.pdf)).

* Use databases (more on this below).

* Use distributed computing (distribute the analysis/simulation across
    multiple CPU’s).

The next subsection superficially covers databases. We will cover memory management in the advanced Python, HPC and C weeks.

### Databases and R

R can be used to link to and extract data from online databases such as PubMed and GenBank, or to manipulate and access your own. Computational Biology datasets are often quite large, and it makes sense to access their data by querying the databases instead of manually downloading them. So also, your own data may be complex and large, in which case you may want to organize and manage those data in a proper relational
database.

Practically all the data wrangling principles in the previous sections are a part and parcel of relational databases.

There are many R packages that provide an interface to databases (SQLite, MySQL, Oracle, etc). Check out R packages `DBI` (<http://cran.r-project.org/web/packages/DBI/index.html>) and ` RMySQL` (<https://cran.r-project.org/web/packages/RMySQL/index.html>.

And just like python (see [these notes](https://github.com/mhasoba/TheMulQuaBio/blob/master/silbiocomp/SilBioComp.pdf)), R can also be used to access, update and manage databases. In particular `SQLite` allows you to build, manipulate, and access databases easily. Try the script called `SQLinR.R` available in the `Code` directory of the master repo. It assumes that you are already familiar with the databases section of the python Chapter in [these notes](https://github.com/mhasoba/TheMulQuaBio/blob/master/silbiocomp/SilBioComp.pdf).

## Readings  & Resources

Check out `DataDataData!` and `R` under [Readings](https://github.com/vectorbite/VBiTraining/tree/master/readings) on this course's repository.

* Brian McGill's [Ten commandments](
    https://dynamicecology.wordpress.com/2016/08/22/ten-commandments-for-good-data-management)

* This paper covers similar ground (available in the readings directory): Borer, E. T., Seabloom, E. W., Jones, M. B., & Schildhauer, M. (2009). Some Simple Guidelines for Effective Data Management. Bulletin of the Ecological Society of America, 90(2), 205–214.

* [Wrangler](http://vis.stanford.edu/papers/wrangler)

* [An interactive framework for data cleaning](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2000/CSD-00-1110.pdf)

* [Wide vs. long data](http://www.theanalysisfactor.com/wide-and-long-data/)